In [ ]:
from datascience import *
%matplotlib inline

import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=np.VisibleDeprecationWarning)

## Lecture 18

## Hypothesis Testing

### Mendel and Pea Flowers

In [ ]:
## Mendel had 929 plants, of which 709 had purple flowers
observed_purples = 709 / 929
observed_purples

In [ ]:
predicted_proportions = make_array(.75, .25)

In [ ]:
sample_proportions(929, predicted_proportions)

In [ ]:
sample_proportions(929, predicted_proportions).item(0)

In [ ]:
# percentage of purple flowers according to distribution
def purple_flowers():
    return sample_proportions(929, predicted_proportions).item(0) * 100

In [ ]:
purple_flowers()

In [ ]:
num_simulations=1000

purples = make_array()

for i in np.arange(num_simulations):
    new_purple = purple_flowers()
    purples = np.append(purples, new_purple)

In [ ]:
Table().with_column('Percent of purple flowers in sample of 929', purples).hist()

In [ ]:
Table().with_column('Discrepancy in sample of 929 if the model is true', abs(purples- 75)).hist()

### How close was Mendel's sample to the model's prediction?

In [ ]:
abs(observed_purples * 100 - 75)

In [ ]:
Table().with_column('Discrepancy in sample of 929 if the model is true', abs(purples- 75)).hist()
# Plotting details; ignore this code
plots.ylim(-0.01, 0.6)
plots.scatter(1.318622174381062, 0, color='red', s=50);

### Simulating jury panels picked at random according to distribution

In [ ]:
sample_size = 100
eligible = [0.26, 0.74]

In [ ]:
sample_proportions(100, eligible)

In [ ]:
sample_proportions(sample_size, eligible).item(0)

In [ ]:
def simulate_one_count():
    return sample_size * sample_proportions(sample_size, eligible).item(0)

In [ ]:
simulate_one_count()

In [ ]:
counts = make_array()
for i in np.arange(10000):
    counts = np.append(counts, simulate_one_count())

In [ ]:
Table().with_column(
    'Count in a Random Sample', counts
).hist(bins = np.arange(5.5, 46.6, 1))

# Plotting details; ignore this code
plots.ylim(-0.002, 0.09)
plots.scatter(8, 0, color='red', s=30);

## Comparing Distributions

<h2> Haribo Gummy Bears </h2>

<h3> In this case, we have more than two categories. How would we calculate a test statistic for that? </h3>

In [ ]:
bears = Table().with_columns(
    'Flavor', make_array('Strawberry', 'Lemon', 'Orange', 'Raspberry', 'Pineapple'),
    'Expected', make_array(0.2, 0.2, 0.2, 0.2, 0.2),
    'Observed', make_array(15/60, 10/60, 7/60, 17/60, 11/60)
)

bears

In [ ]:
bears.barh('Flavor')

In [ ]:
model = make_array(0.2, 0.2, 0.2, 0.2, 0.2)
model

In [ ]:
simulation = sample_proportions(60, model)
simulation

In [ ]:
bears.with_column("Simulated", simulation).barh("Flavor")

<h2> Distance Between Distributions </h2>

In [ ]:
# Earlier, the difference between observed purple flowers
# and the expected values (75%) was our statistic.
#
# In this case, we need to understand how each of the 5 categories
# differs from its expected value according to the model.

diffs = bears.column('Observed') - bears.column('Expected')
bears_with_difference = bears.with_column('Difference', diffs)
bears_with_difference

In [ ]:
np.sum(bears_with_difference.column('Difference'))

This is zero, for all intents and purposes. 

In [ ]:
np.sum(abs(bears_with_difference.column('Difference')))/2

Why do we divide by 2?

In [ ]:
example1 = Table().with_columns(
    'Example', make_array('Category 1', 'Category 2', 'Category 3'),
    'Expected', make_array(1/3, 1/3, 1/3),
    'Observed', make_array(1/3, 1/3, 1/3)
)
example1

In [ ]:
sum(abs(example1.column('Observed') -  example1.column('Expected')))

In [ ]:
example2 = Table().with_columns(
    'Example', make_array('Category 1', 'Category 2', 'Category 3'),
    'Expected', make_array(0, 1, 0),
    'Observed', make_array(1, 0, 0)
)
example2

In [ ]:
sum(abs(example2.column('Observed') -  example2.column('Expected')))

In [ ]:
bears_with_difference

In [ ]:
np.sum(abs(bears_with_difference.column("Difference")))

In [ ]:
0.05 + 0.0833333 #sum of positive entries

In [ ]:
-0.0333333 - 0.0833333 - 0.0166667 # sum of negative entries

In [ ]:
np.sum(abs(bears_with_difference.column('Difference')))/2

## Total Variation Distance -- TVD

In [ ]:
def tvd(dist1, dist2):
    return sum(abs(dist1 - dist2))/2

In [ ]:
# The TVD of our observed data from their expected values
# assuming the model is true
observed_stat = tvd(bears.column('Observed'), bears.column('Expected'))
observed_stat

Same value we got previously. 

<h3>Let's do the same comparison with the simulated distribution:</h3>

In [ ]:
odel = make_array(0.2, 0.2, 0.2, 0.2, 0.2)

In [ ]:
sample_proportions(60, model)

In [ ]:
# The TVD of a model simluation from its expected values
tvd(sample_proportions(60, model), bears.column('Expected'))

In [ ]:
def simulated_tvd():
    return tvd(sample_proportions(60, model), model)

In [ ]:
tvds = make_array()

num_simulations = 10000
for i in np.arange(num_simulations):
    new_tvd = simulated_tvd()
    tvds = np.append(tvds, new_tvd)
tvds

In [ ]:
title = 'Simulated TVDs (if model is true)'

Table().with_column(title, tvds).hist()
plots.scatter(0.13333333333, 0, color='red', s=200);
print('Observed TVD: ' + str(observed_stat))